# imports

In [1]:
import os, random, struct
from Crypto.Cipher import AES
import hashlib
from Crypto.Cipher import PKCS1_OAEP
import binascii
from Crypto.PublicKey import RSA

# create account

In [2]:
def number_account(n):
    return int(input("input number of account :"))

In [3]:
def list_name_account(number_account):
                      list_name_account=[]
                      for i in range(number_account):
                            list_name_account.append(input("input account name :"))
                      return list_name_account

In [4]:
def create_account(list_name_account,root_path):
                folders = list_name_account
                folders.append('public_key')
                for folder in folders:
                      os.mkdir(os.path.join(root_path,folder))

# padding file AES

In [5]:
def pad_f(file):
    while len(file) % 16 != 0:
        file=file + b'0'
    return file

# key generation 

In [6]:
def key_genration_RSA(list_name_account,pub_key_path,private_key_path):
    for i in list_name_account:
        password =  input("input your password :")   
        key_aes  =  hashlib.sha256(password.encode(encoding='UTF-8')).digest()
        IV       =  os.urandom(16)
        keyPair  =  RSA.generate(7680)
        privKey  =  keyPair.exportKey('PEM')  
        pubKey   =  keyPair.publickey().exportKey('PEM')
        mode=AES.MODE_CBC
        cipher = AES.new(key_aes,mode,IV)
        with open(pub_key_path+i+'.pem', 'wb') as file:  
                file.write(pubKey)
        with open(private_key_path+i+'/'+i+'.pem', 'wb') as file:  
                file.write(privKey)
        with open(private_key_path+i+'/'+i+'.pem',"rb") as f:
              orig_file=f.read()
        padded_file= pad_f(orig_file)
        filesize = os.path.getsize(private_key_path+i+'/'+i+'.pem')
        with open(private_key_path+i+'/'+i+'.txt', 'wb') as outfile:
            outfile.write(struct.pack('<Q', filesize))
            outfile.write(IV)
            outfile.write(cipher.encrypt(padded_file))
        with open(private_key_path+i+'/'+i+'.pem', 'wb') as file:  
                file.write(os.urandom(int(filesize)))
        os.remove(private_key_path+i+'/'+i+'.pem')

# encryption_file

In [7]:
def aes_encryption(in_file_path_for_encryption,out_file_encryption,public_key_path):
    Key=hashlib.sha256(os.urandom(32)).digest()
    IV=os.urandom(16)
    mode=AES.MODE_CBC
    cipher=AES.new(Key,mode,IV)
    with open(in_file_path_for_encryption,"rb") as f:
          orig_file=f.read()
    padded_file= pad_f(orig_file)
    f1 = open(public_key_path, 'rb')  
    pubKey = RSA.importKey(f1.read())  
    encryptor = PKCS1_OAEP.new(pubKey)
    encrypted_key = encryptor.encrypt(Key)
    with open(out_file_encryption+"key"+'.txt', 'wb') as outfile:
            outfile.write(encrypted_key)
    filesize_key=os.path.getsize(out_file_encryption+"key"+'.txt')
    os.remove(out_file_encryption+"key"+'.txt')
    filesize = os.path.getsize(in_file_path_for_encryption)
    with open(out_file_encryption+"encryption"+'.txt', 'wb') as outfile:
            outfile.write(struct.pack('<Q', filesize))
            outfile.write(struct.pack('<Q', filesize_key))
            outfile.write(IV)
            outfile.write(encrypted_key)
            outfile.write(cipher.encrypt(padded_file))

# decryption_file

In [8]:
def aes_decryption(in_file_ciphertext,out_file_decryption,priv_key_path):
    password =  input("input your password :")   
    key_aes  =  hashlib.sha256(password.encode(encoding='UTF-8')).digest()
    mode=AES.MODE_CBC
    with open(priv_key_path, 'rb') as infile:
        origsize1 = struct.unpack('<Q', infile.read(struct.calcsize('Q')))[0]
        IV = infile.read(16)
        decryptor=AES.new(key_aes,mode,IV)
        with open(out_file_decryption+'p'+'.pem', 'wb') as file:  
                file.write(decryptor.decrypt(infile.read()))
                file.truncate(origsize1)
    with open(in_file_ciphertext, 'rb') as infile:
        origsize = struct.unpack('<Q', infile.read(struct.calcsize('Q')))[0]
        key_size = struct.unpack('<Q', infile.read(struct.calcsize('Q')))[0]
        iv = infile.read(16)
        secretkey=infile.read(key_size)#960
        f = open(out_file_decryption+'p'+'.pem', 'rb')  
        key = RSA.importKey(f.read())
        decryptor = PKCS1_OAEP.new(key)
        decrypted_aes_key = decryptor.decrypt(secretkey)
        mode=AES.MODE_CBC
        decryptor=AES.new(decrypted_aes_key,mode,iv)
        with open(out_file_decryption+"decryption"+'.xlsx','wb') as outfile:
                 outfile.write(decryptor.decrypt(infile.read()))
                 outfile.truncate(origsize)
    with  open(out_file_decryption+'p'+'.pem', 'wb') as file:  
                file.write(os.urandom(int(origsize1)))
    #file.close()
    os.remove(out_file_decryption+'p'+'.pem')

#ciphertext_path
in_file_ciphertext='C:/Users/User/Desktop/file_accuont/ciphertext/encryption.txt'
# decryption_path
out_file_decryption='C:/Users/User/Desktop/file_accuont/decryption/'
# private_key_path
priv_key_path='C:/Users/User/Desktop/file_accuont/a/a.txt'
#aes_decryption(in_file_ciphertext,out_file_decryption,priv_key_path)
